In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras import utils
utils.to_categorical

from difflib import SequenceMatcher
from tensorflow.keras.utils import to_categorical

In [2]:
# Membaca dataset utama untuk pelatihan
data_df = pd.read_csv('E:/Sekolah/Telkom/TA/LTSM/Process/Comb/All.csv')

In [3]:
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values

In [4]:
print(y)

['Density 30%' 'Density 30%' 'Density 30%' ... 'Density 90%' 'Density 90%'
 'Density 90%']


In [5]:
# Menggunakan LabelEncoder untuk mengubah label menjadi nilai numerik
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded_categorical = utils.to_categorical(y_encoded)
print(y_encoded_categorical.shape)

(599040, 4)


In [6]:
# Membagi dataset menjadi data pelatihan dan data validasi
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded_categorical, test_size=0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

In [7]:
print(X_val.shape)

(119808, 2, 1)


In [8]:
# Membangun model LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_encoded_categorical.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
# Melatih model dengan dataset utama
# model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val))

In [10]:
# model.save('All.h5')

In [11]:
model = load_model('All.h5')

In [12]:
# Membaca dataset yang ingin diprediksi
predict_df = pd.read_csv('E:/Sekolah/Telkom/TA/LTSM/Process/50%/K502_processed.csv')

In [13]:
# Memisahkan fitur (X) dari data yang ingin diprediksi
X_predict = predict_df.iloc[:].values

In [14]:
# Memprediksi label menggunakan model yang telah dilatih
y_pred_encoded = model.predict(X_predict)
y_pred = np.argmax(y_pred_encoded, axis=1)
y_pred_label = label_encoder.inverse_transform(y_pred)

390/390 [==============================] - 3s 4ms/step


In [15]:
print(y_pred_label)

['Density 70%' 'Density 70%' 'Density 70%' ... 'Density 30%' 'Density 70%'
 'Density 70%']


In [16]:
result_df = predict_df.copy()
result_df['predicted_label'] = y_pred_label

In [17]:
result_df.to_csv('data_predict_predicted.csv', index=False)

In [18]:
# Menampilkan kategori label dari dataset yang diprediksi
print("Kategori Label dari Dataset yang Diprediksi:")
print(result_df['predicted_label'].value_counts())

Kategori Label dari Dataset yang Diprediksi:
predicted_label
Density 50%    8802
Density 90%    2710
Density 30%     591
Density 70%     377
Name: count, dtype: int64


In [21]:
# Membandingkan kemiripan label dengan dataset utama
similar_labels = []
for label in result_df['predicted_label'].unique():
    similarity = max(SequenceMatcher(None, label, unique_label).ratio() for unique_label in np.unique(y))
if similarity > 0.7:
    print("Data match :", label)

Data match : Density 50%


In [20]:
print(similar_labels)

[]
